In [1]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [3]:
lemmatizer=WordNetLemmatizer()

In [4]:
with open("intents.json", "r", encoding="utf-8") as f:
    intents = json.load(f)


In [5]:
word=[]
documents = []
classes = []
ignore_letters=["?","!",",","."]


In [6]:

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        word_list = nltk.word_tokenize(pattern)
        word.append(word_list)
        documents.append((word_list, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])
print(documents)

[(['hello'], 'greetings'), (['hey'], 'greetings'), (['hi'], 'greetings'), (['good', 'day'], 'greetings'), (['Greetings'], 'greetings'), (['What', "'s", 'up', '?'], 'greetings'), (['how', 'is', 'it', 'going', '?'], 'greetings'), (['cya'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Have', 'a', 'good', 'day'], 'goodbye'), (['bye'], 'goodbye'), (['cao'], 'goodbye'), (['see', 'ya'], 'goodbye'), (['how', 'old'], 'age'), (['้how', 'old', 'is', 'Phad'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['how', 'old', 'are', 'you'], 'age'), (['age', '?'], 'age'), (['what', 'is', 'your', 'name'], 'name'), (['what', 'should', 'I', 'call', 'you'], 'name'), (['whats', 'your', 'name', '?'], 'name'), (['who', 'are', 'you', '?'], 'name'), (['What', 'is', 'your', 'religion', '?'], 'religion'), (['what', 'is', 'your', 'attitude', '?'], 'attitude')]


In [7]:

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        word_list = nltk.word_tokenize(pattern)
        word.extend(word_list)
        documents.append((word_list, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])


In [8]:
words = [lemmatizer.lemmatize(word) for word in word_list if word not in ignore_letters]
words = sorted(set(words))
print(words)


['attitude', 'is', 'what', 'your']


In [9]:
classes=sorted(set(classes))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [10]:
training=[]
output_empty=[0]*len(classes)

for document in documents:
    bag=[]
    word_patterns=document[0]
    word_patterns=[lemmatizer.lemmatize(word.lower())for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    

    output_row=list(output_empty)
    output_row[classes.index(document[1])] =1
    training.append([bag, output_row])


In [11]:
random.shuffle(training)
training = [(row[0], row[1]) for row in training] # convert to list of tuples

train_x = [row[0] for row in training] # extract bag of words
train_y = [row[1] for row in training] # extract output row


In [12]:
model=Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),),activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation="softmax"))

In [13]:
from tensorflow.keras.optimizers import SGD

sgd = SGD(learning_rate=0.09, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

hist=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

model.save("chatbot_model.h5",hist)
print("Done")

Epoch 1/200
11/11 [==============================] - 0s 2ms/step - loss: 1.7675 - accuracy: 0.1731
Epoch 2/200
11/11 [==============================] - 0s 1ms/step - loss: 1.5607 - accuracy: 0.3269
Epoch 3/200
11/11 [==============================] - 0s 2ms/step - loss: 1.8826 - accuracy: 0.2885
Epoch 4/200
11/11 [==============================] - 0s 289us/step - loss: 1.7830 - accuracy: 0.3077
Epoch 5/200
11/11 [==============================] - 0s 1ms/step - loss: 1.6970 - accuracy: 0.2692
Epoch 6/200
11/11 [==============================] - 0s 1ms/step - loss: 1.6266 - accuracy: 0.2885
Epoch 7/200
11/11 [==============================] - 0s 2ms/step - loss: 1.6100 - accuracy: 0.3269
Epoch 8/200
11/11 [==============================] - 0s 2ms/step - loss: 1.6996 - accuracy: 0.2500
Epoch 9/200
11/11 [==============================] - 0s 2ms/step - loss: 1.5776 - accuracy: 0.2692
Epoch 10/200
11/11 [==============================] - 0s 499us/step - loss: 1.5579 - accuracy: 0.2692
Epoch